In [175]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt

In [75]:
user_engage = pd.read_csv('takehome_user_engagement.csv')

In [76]:
user_engage

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1
...,...,...,...
207912,2013-09-06 06:14:15,11996,1
207913,2013-01-15 18:28:37,11997,1
207914,2014-04-27 12:45:16,11998,1
207915,2012-06-02 11:55:59,11999,1


In [77]:
users = pd.read_csv('takehome_users.csv')

In [78]:
users

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0
...,...,...,...,...,...,...,...,...,...,...
11995,11996,2013-09-06 06:14:15,Meier Sophia,SophiaMeier@gustr.com,ORG_INVITE,1.378448e+09,0,0,89,8263.0
11996,11997,2013-01-10 18:28:37,Fisher Amelie,AmelieFisher@gmail.com,SIGNUP_GOOGLE_AUTH,1.358275e+09,0,0,200,NaN
11997,11998,2014-04-27 12:45:16,Haynes Jake,JakeHaynes@cuvox.de,GUEST_INVITE,1.398603e+09,1,1,83,8074.0
11998,11999,2012-05-31 11:55:59,Faber Annett,mhaerzxp@iuxiw.com,PERSONAL_PROJECTS,1.338638e+09,0,0,6,NaN


In [112]:
users.rename(columns= {'object_id': 'user_id'}, inplace= True)

In [79]:
print(user_engage.info())
print(users.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time_stamp  207917 non-null  object
 1   user_id     207917 non-null  int64 
 2   visited     207917 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   object_id                   12000 non-null  int64  
 1   creation_time               12000 non-null  object 
 2   name                        12000 non-null  object 
 3   email                       12000 non-null  object 
 4   creation_source             12000 non-null  object 
 5   last_session_creation_time  8823 non-null   float64
 6   opted_in_to_mailing_list    12000 non-null  int64  
 7   ena

In [80]:
user_engage.isnull().any()

time_stamp    False
user_id       False
visited       False
dtype: bool

In [81]:
user_engage['time_stamp'] = pd.to_datetime(user_engage['time_stamp'])

In [82]:
user_engage.set_index('time_stamp', inplace= True)

In [83]:
users['creation_time'] = pd.to_datetime(users['creation_time'])

In [84]:
users.isnull().any()

object_id                     False
creation_time                 False
name                          False
email                         False
creation_source               False
last_session_creation_time     True
opted_in_to_mailing_list      False
enabled_for_marketing_drip    False
org_id                        False
invited_by_user_id             True
dtype: bool

In [85]:
users.last_session_creation_time.isnull().values.sum()

3177

In [86]:
users.invited_by_user_id.isnull().values.sum()

5583

In [87]:
users['last_session_creation_time'] = pd.to_datetime(users['last_session_creation_time'], unit= 's', errors= 'coerce')

In [88]:
users.sample(5)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
10932,10933,2014-01-20 09:47:06,Sousa Pedro,PedroGomesSousa@cuvox.de,ORG_INVITE,2014-01-21 09:47:06,1,1,82,3138.0
6461,6462,2013-10-23 04:30:37,Gottlieb Maik,MaikGottlieb@gmail.com,SIGNUP_GOOGLE_AUTH,2014-06-04 04:30:37,0,0,248,NaN
107,108,2013-04-16 20:10:27,Pabst Kristin,KristinPabst@jourrapide.com,GUEST_INVITE,NaT,1,0,306,3522.0
6500,6501,2014-05-26 12:13:54,Lucas Andrew,AndrewSLucas@gmail.com,SIGNUP_GOOGLE_AUTH,2014-05-26 12:13:54,0,0,13,NaN
5746,5747,2012-09-10 06:33:30,Ribeiro Samuel,SamuelOliveiraRibeiro@jourrapide.com,PERSONAL_PROJECTS,NaT,0,0,213,NaN


We can see that both creation_time and last_session_creation_time match in values. Therefore, will fill the NaN values of the latter with the former

In [89]:
# Fill NaN values in last_session_creation_time

users.last_session_creation_time.fillna(users.creation_time, inplace= True)

In [90]:
# Fill the NaN values for invited_by_user_id with a placeholder. We shall keep this cause it's still considered valueable data if
# the user was not invited by anyone else.

users.invited_by_user_id.fillna('None', inplace= True)

In [51]:
users.isnull().any()

user_id                       False
creation_time                 False
name                          False
email                         False
creation_source               False
last_session_creation_time    False
opted_in_to_mailing_list      False
enabled_for_marketing_drip    False
org_id                        False
invited_by_user_id            False
dtype: bool

In [91]:
# This function will find which users have been active in any given 7 day period

def adopted_users(df, days= 7, logins= 3):
    df['date'] = df.index.date
    df.drop_duplicates(subset= 'date').sort_values('date')
    passed_days = df['date'].diff(periods= logins - 1)
    return any(passed_days <= timedelta(days= days))

adopted_user = user_engage.groupby('user_id').apply(adopted_users)
adopted_user.name = 'adopted_user'

In [92]:
adopted_user.head()

user_id
1    False
2     True
3    False
4    False
5    False
Name: adopted_user, dtype: bool

In [93]:
adopted_user_df = pd.DataFrame(columns= ['user_id', 'adopted_user'])

In [94]:
adopted_user_df.user_id = np.arange(len(adopted_user))
adopted_user_df.adopted_user = adopted_user

In [95]:
adopted_user_df.head()

,user_id,adopted_user
0,0,NaN
1,1,False
2,2,True
3,3,False
4,4,False


In [153]:
adopted_user.dropna()

user_id
1        False
2         True
3        False
4        False
5        False
         ...  
11996    False
11997    False
11998    False
11999    False
12000    False
Name: adopted_user, Length: 8823, dtype: bool

In [161]:
relax_df = pd.merge(adopted_user_df, users)

In [162]:
relax_df.fillna(False, inplace= True)

In [163]:
relax_df.isnull().any()

user_id                       False
adopted_user                  False
creation_time                 False
name                          False
email                         False
creation_source               False
last_session_creation_time    False
opted_in_to_mailing_list      False
enabled_for_marketing_drip    False
org_id                        False
invited_by_user_id            False
dtype: bool

In [165]:
relax_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8822 entries, 0 to 8821
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   user_id                     8822 non-null   int32         
 1   adopted_user                8822 non-null   bool          
 2   creation_time               8822 non-null   datetime64[ns]
 3   name                        8822 non-null   object        
 4   email                       8822 non-null   object        
 5   creation_source             8822 non-null   object        
 6   last_session_creation_time  8822 non-null   datetime64[ns]
 7   opted_in_to_mailing_list    8822 non-null   int64         
 8   enabled_for_marketing_drip  8822 non-null   int64         
 9   org_id                      8822 non-null   int64         
 10  invited_by_user_id          8822 non-null   object        
dtypes: bool(1), datetime64[ns](2), int32(1), int64(3), objec

In [166]:
y = relax_df['adopted_user']
x = relax_df.drop(['adopted_user', 'creation_time','name','user_id','email','last_session_creation_time'], axis= 1)

x = pd.get_dummies(x)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 123)

In [177]:
rfc = RandomForestClassifier(n_estimators= 100)

rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)



In [168]:
metrics.accuracy_score(y_test, y_pred)

0.8175637393767705

In [169]:
featureImp = pd.Series(rfc.feature_importances_, index = x.columns.tolist()).sort_values(ascending = False)
featureImp

org_id                          0.373960
opted_in_to_mailing_list        0.028380
enabled_for_marketing_drip      0.020748
creation_source_ORG_INVITE      0.013115
creation_source_GUEST_INVITE    0.012556
                                  ...   
invited_by_user_id_5079.0       0.000000
invited_by_user_id_10334.0      0.000000
invited_by_user_id_2195.0       0.000000
invited_by_user_id_11144.0      0.000000
invited_by_user_id_7468.0       0.000000
Length: 2231, dtype: float64

It can be seen that the organization id, in which the users are grouped together by organiations, is a major factor in the adoption of users. This makes sense as if it is the mode of operations for certain organizations to use the services then the employees will have to use it to as well to continue tp fulfill their job tasks.